In [1]:
from pyscipopt import Model,quicksum, multidict
import numpy as np

In [2]:
model = Model("Example")

In [3]:
M =5
N =10
L = [10,19,91,18,14]
S = [14,14,18,20,25]
D = [2,4,3,8,3,8,3,4,5,5]
C = []  #单位运输费用
for i in range(5):
    C.append([])
    for j in range(10):
        C[i].append(1)
C = np.array(C)
C

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [4]:
Q ={}
for i in range(1,M+1):
    for j in range(1,N+1):
        Q[i,j] = model.addVar(vtype="C", name="Q(%s,%s)"%(i,j))

In [5]:
X = {}
for i in range(1,M+1):
    for j in range(1,N+1):
        X[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j))

In [6]:
for i in range(1,M+1):
    model.addCons(quicksum(Q[i,j] for j in range(1,N+1)) <= S[i-1], "Out(%s)"%i)

In [7]:
for j in range(1,N+1):
    model.addCons(quicksum(Q[i,j] for i in range(1,M+1)) >= D[i-1], "Out(%s)"%j)

In [8]:
for i in range(1,M+1):
    for j in range(1,N+1):
        model.addCons(1000* X[i,j] >= Q[i, j], "MTZ(%s,%s)"%(i,j))
        model.addCons(X[i,j] <= Q [i,j],"MTZ(%s,%s)"%(i,j))

In [9]:
for i in range(1, M+1):
    model.addCons(quicksum(X[i, j] for j in range(1, N+1))<= L[i-1],"Out(%s)"%i)

In [10]:
 model.setObjective(quicksum(C[i-1,j-1]*Q[i,j] for (i,j) in Q), "minimize")

In [11]:
model.optimize()

In [ ]:
 model.data = Q

In [ ]:
print("obj:", model.getObjVal())
print("gap:", model.getGap())
for v in model.getVars():
    if model.getVal(v)> 0:
        print(v.name, "=", model.getVal(v))